# Validating measures

In [1]:
import pandas as pd
import numpy as np
import pickle as pkl
import matplotlib.pyplot as plt
import time

### Data Directory

In [2]:
INPUT_DIR = '../data/interim/'
OUTPUT_DIR = '../data/processed/'

### Reading predicted file

In [3]:
ip_file = INPUT_DIR + 'fd002_dbscan_results_1000_0.001.csv'
df_pred = pd.read_csv(ip_file, sep=',')

In [4]:
df_pred.head()

,setting1,setting2,setting3,meas01,meas02,meas03,meas04,meas05,meas06,meas07,...,meas15,meas16,meas17,meas18,meas19,meas20,meas21,unit,time_cycles,labels
0,34.9983,0.8400,100.0,449.44,555.32,1358.61,1137.23,5.48,8.00,194.64,...,9.3461,0.02,334,2223,100.00,14.73,8.8071,1,1,0
1,41.9982,0.8408,100.0,445.00,549.90,1353.22,1125.78,3.91,5.71,138.51,...,9.3774,0.02,330,2212,100.00,10.41,6.2665,1,2,1
2,24.9988,0.6218,60.0,462.54,537.31,1256.76,1047.45,7.05,9.02,175.71,...,10.8941,0.02,309,1915,84.93,14.08,8.6723,1,3,2
3,42.0077,0.8416,100.0,445.00,549.51,1354.03,1126.38,3.91,5.71,138.46,...,9.3528,0.02,329,2212,100.00,10.59,6.4701,1,4,1
4,25.0005,0.6203,60.0,462.54,537.07,1257.71,1047.93,7.05,9.03,175.05,...,10.8963,0.02,309,1915,84.93,14.13,8.5286,1,5,2


In [5]:
# Make everything in labels column into 0 or -1
df_pred['labels'] = df_pred['labels'].apply(lambda x: 0 if x >= 0 else -1)

### Generating Ground Truth with heuristics

In [6]:
timecycles_for_failure = 10

In [7]:
df_pred['ground_truth'] = 0

In [8]:
df_pred.head()

,setting1,setting2,setting3,meas01,meas02,meas03,meas04,meas05,meas06,meas07,...,meas16,meas17,meas18,meas19,meas20,meas21,unit,time_cycles,labels,ground_truth
0,34.9983,0.8400,100.0,449.44,555.32,1358.61,1137.23,5.48,8.00,194.64,...,0.02,334,2223,100.00,14.73,8.8071,1,1,0,0
1,41.9982,0.8408,100.0,445.00,549.90,1353.22,1125.78,3.91,5.71,138.51,...,0.02,330,2212,100.00,10.41,6.2665,1,2,0,0
2,24.9988,0.6218,60.0,462.54,537.31,1256.76,1047.45,7.05,9.02,175.71,...,0.02,309,1915,84.93,14.08,8.6723,1,3,0,0
3,42.0077,0.8416,100.0,445.00,549.51,1354.03,1126.38,3.91,5.71,138.46,...,0.02,329,2212,100.00,10.59,6.4701,1,4,0,0
4,25.0005,0.6203,60.0,462.54,537.07,1257.71,1047.93,7.05,9.03,175.05,...,0.02,309,1915,84.93,14.13,8.5286,1,5,0,0


In [9]:
units = np.unique(df_pred['unit'])

In [10]:
df_pred.loc[df_pred['unit'] == -1]

,setting1,setting2,setting3,meas01,meas02,meas03,meas04,meas05,meas06,meas07,...,meas16,meas17,meas18,meas19,meas20,meas21,unit,time_cycles,labels,ground_truth


In [11]:
# Assign outliers for the last 'n' time cycles for each of the unit

units = np.unique(df_pred['unit'])
# print('Number of unique units:', len(units))
for current_unit in units:
    df_sub = df_pred.loc[df_pred['unit'] == current_unit]
    max_timecycle = max(df_sub['time_cycles'])    
    outlier_time_threshold = max_timecycle - timecycles_for_failure
    
    # print('Unit=', current_unit, 'outlier threshold', outlier_time_threshold)
    
    df_pred['ground_truth'].loc[(df_pred['unit'] == current_unit) & (df_pred['time_cycles'] > outlier_time_threshold)] = -1

/Users/talat/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [12]:
print(df_pred[['unit', 'time_cycles', 'labels', 'ground_truth']].loc[df_pred['unit'] == 1])

     unit  time_cycles  labels  ground_truth
0       1            1       0             0
1       1            2       0             0
2       1            3       0             0
3       1            4       0             0
4       1            5       0             0
5       1            6       0             0
6       1            7       0             0
7       1            8       0             0
8       1            9       0             0
9       1           10       0             0
10      1           11       0             0
11      1           12       0             0
12      1           13       0             0
13      1           14       0             0
14      1           15       0             0
15      1           16       0             0
16      1           17       0             0
17      1           18       0             0
18      1           19       0             0
19      1           20       0             0
20      1           21       0             0
21      1 

In [13]:
print('Number of outliers in the predicted values', len(df_pred.loc[df_pred['labels'] == -1]))
print('Number of outliers in the ground truth', len(df_pred.loc[df_pred['ground_truth'] == -1]))

Number of outliers in the predicted values 1052
Number of outliers in the ground truth 2600


In [14]:
# Copy into a new dataframe with only the required columns
df_scoring = df_pred[['unit', 'time_cycles', 'labels', 'ground_truth']].copy()

In [15]:
# tp = 0
# fp = 1
# fn = 2
# tn = 3

def gen_vals(ground_truth, pred_label, pos_label, neg_label):
    if (ground_truth == pos_label) & (pred_label == pos_label):
        return 0
    elif (ground_truth == neg_label) & (pred_label == pos_label):
        return 1
    elif (ground_truth == pos_label) & (pred_label == neg_label):
        return 2
    elif (ground_truth == neg_label) & (pred_label == neg_label):
        return 3

### Find out the f-measure

In [16]:
pos_label = -1
neg_label = 0

df_scoring['pos'] = -1

df_scoring['pos'] = df_scoring.apply(lambda x: gen_vals(x['ground_truth'], x['labels'], pos_label, neg_label), axis=1)

In [17]:
from collections import Counter
counter = Counter(df_scoring['pos'])

print(counter)

accuracy = (counter[0] + counter[3]) / (counter[0] + counter[1] + counter[2] + counter[3])
print('Accuracy = ', accuracy)

precision = counter[0] / (counter[0] + counter[1])
print('Precision = ', precision)

recall = counter[0] / (counter[0] + counter[2])
print('Recall = ', recall)

f1_score = 2 * (precision * recall) / (precision + recall)
print('F1 score: ', f1_score)

Counter({3: 50694, 2: 2013, 0: 587, 1: 465})
Accuracy =  0.9539053925854275
Precision =  0.5579847908745247
Recall =  0.22576923076923078
F1 score:  0.3214676889375684


# Making it into a function

In [18]:
def find_eval_metrics(df_pred, timecycles_for_failure):

    # Make everything in labels column into 0 or -1
    df_pred['labels'] = df_pred['labels'].apply(lambda x: 0 if x >= 0 else -1)
    
    df_pred['ground_truth'] = 0
    
    # Assign outliers for the last 'n' time cycles for each of the unit
    units = np.unique(df_pred['unit'])
    # print('Number of unique units:', len(units))
    for current_unit in units:
        df_sub = df_pred.loc[df_pred['unit'] == current_unit]
        max_timecycle = max(df_sub['time_cycles'])    
        outlier_time_threshold = max_timecycle - timecycles_for_failure

        # print('Unit=', current_unit, 'outlier threshold', outlier_time_threshold)

        df_pred['ground_truth'].loc[(df_pred['unit'] == current_unit) & (df_pred['time_cycles'] > outlier_time_threshold)] = -1
        
    # Copy into a new dataframe with only the required columns
    df_scoring = df_pred[['unit', 'time_cycles', 'labels', 'ground_truth']].copy()
    
    pos_label = -1
    neg_label = 0

    df_scoring['pos'] = -1
    df_scoring['pos'] = df_scoring.apply(lambda x: gen_vals(x['ground_truth'], x['labels'], pos_label, neg_label), axis=1)
    
    counter = Counter(df_scoring['pos'])

    accuracy = (counter[0] + counter[3]) / (counter[0] + counter[1] + counter[2] + counter[3])
    # print('Accuracy = ', accuracy)
    precision = counter[0] / (counter[0] + counter[1])
    # print('Precision = ', precision)
    recall = counter[0] / (counter[0] + counter[2])
    # print('Recall = ', recall)
    f1_score = 2 * (precision * recall) / (precision + recall)
    # print('F1 score: ', f1_score)
    
    return accuracy, precision, recall, f1_score

In [ ]:
import warnings
warnings.filterwarnings("ignore")

max_timecycles_for_failure = 40

print('| timecycles_for_failure|', 'accuracy', '|', 'precision', '|', 'recall', '|', 'f1_score', '|')
print('|--|--|--|--|--|')

for i in range(1, max_timecycles_for_failure+1):
    accuracy, precision, recall, f1_score = find_eval_metrics(df_pred, i)
    print('|', i, '|', accuracy, '|', precision, '|', recall, '|', f1_score, '|')

| timecycles_for_failure| accuracy | precision | recall | f1_score |
|--|--|--|--|--|
| 1 | 0.9785710299670752 | 0.07604562737642585 | 0.3076923076923077 | 0.12195121951219512 |
| 2 | 0.9763760486616194 | 0.1435361216730038 | 0.2903846153846154 | 0.19211195928753183 |
| 3 | 0.9739950519912944 | 0.20627376425855512 | 0.2782051282051282 | 0.2368995633187773 |
| 4 | 0.9715396491750219 | 0.2671102661596958 | 0.2701923076923077 | 0.26864244741873805 |
| 5 | 0.9690470432857754 | 0.3269961977186312 | 0.26461538461538464 | 0.29251700680272114 |
| 6 | 0.9661824066667907 | 0.3773764258555133 | 0.25448717948717947 | 0.30398162327718226 |
| 7 | 0.9634293792667274 | 0.4306083650190114 | 0.2489010989010989 | 0.3154596100278552 |
| 8 | 0.9604159303558474 | 0.47718631178707227 | 0.24134615384615385 | 0.3205619412515964 |
| 9 | 0.9573280752990198 | 0.5218631178707225 | 0.23461538461538461 | 0.32370283018867924 |
| 10 | 0.9539053925854275 | 0.5579847908745247 | 0.22576923076923078 | 0.3214676889375684 |